### Chargement du dataset

In [2]:
csv_pvf10 = "../data/processed/structure_pvf_10.csv"

In [3]:
import pandas as pd
import numpy as np

# Charger le dataframe à partir du dataset
def load_df_from_dataset(csv_path,ensemble,format):
    # Charger le fichier CSV
    df_pvf10 = pd.read_csv(csv_path)
    # Filtrer par ensemble (train/test) et par format
    df_pvf10 = df_pvf10.loc[(df_pvf10['Train_Test'] == ensemble) & (df_pvf10['Format'] == format)]   
    return df_pvf10

# Chargement dataframes d'entraînement et de test
df_train = load_df_from_dataset(csv_pvf10, 'train','110x60')
df_test = load_df_from_dataset(csv_pvf10, 'test','110x60')

# Normalisation des chemins d'accès
df_train['Chemin'] = df_train['Chemin'].str.replace("\\", "/", regex=False)
df_test['Chemin'] = df_test['Chemin'].str.replace("\\", "/", regex=False)

# Séparation features / cible
X_train = df_train.drop('Classe',axis=1)
y_train = df_train['Classe']
X_test = df_test.drop('Classe',axis=1)
y_test = df_test['Classe']

### Pipeline 1 : vecteurs HOG

#### Transformeur pour extraction des vecteurs HOG

In [4]:
import cv2
from skimage.feature import hog
from sklearn.base import BaseEstimator, TransformerMixin

# Transformeur qui renvoie le dataframe des vecteurs HOG
class HOGExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, image_size=(60,110), pixels_per_cell=(8,8),cells_per_block=(2,2)):
        self.image_size = image_size
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.feature_names_ = None

    # Méthode d'extraction du vecteur HOG d'une image
    def extract_hog(self, gray_img):
        hog_vector = hog(
            gray_img,
            orientations=9,
            pixels_per_cell=self.pixels_per_cell,
            cells_per_block=self.cells_per_block,
            feature_vector=True
        )
        return hog_vector
    
    # Le fit ne sert qu'à récupérer le nom des features créées
    def fit(self, X, y=None):
        # On utilise une seule image
        img = cv2.imread(X['Chemin'].iloc[0])
        # Conversion en niveaux de gris et resizing
        img_gray_resized = cv2.cvtColor(cv2.resize(img, self.image_size), cv2.COLOR_BGR2GRAY)
        # Extraction du vecteur HOG
        hog_vector = self.extract_hog(img_gray_resized)
        # On en déduit le nom des features
        self.feature_names_ =  [f'HOG_{i+1}' for i in range(len(hog_vector))]
        return self
    
    def transform(self, X):
        # On va calculer les vecteurs HOG
        hog_vectors = []
        # On parcourt les chemins des images
        for img_path in X['Chemin']:
            # Lecture de l'image
            img = cv2.imread(img_path)
            # Conversion en niveaux de gris et resizing
            img_gray_resized = cv2.cvtColor(cv2.resize(img, self.image_size), cv2.COLOR_BGR2GRAY)
            # Extraction et stockage du vecteur HOG
            hog_vector = self.extract_hog(img_gray_resized)
            hog_vectors.append(hog_vector)
        # On renvoie le dataframe des vecteurs HOG
        return pd.DataFrame(hog_vectors,index=X.index,columns=self.feature_names_)
    
    # Pour récupération du nom des features créées
    def get_feature_names_out(self, input_features=None):
        return self.feature_names_
    
# Création du transformeur
hog_extr = HOGExtractor()

#### Standardisation

In [5]:
from sklearn.preprocessing import StandardScaler

# Création du StandardScaler
hog_st = StandardScaler()


#### Création Pipeline HOG

In [6]:
from sklearn.pipeline import Pipeline

HOGPipeline = Pipeline(steps=[
    ("Extraction HOG",hog_extr),
    ("Standardisation",hog_st)
]) 

### Pipeline 2 : vecteurs GLCM 

#### Transformeur pour extraction des vecteurs GLCM

In [7]:
from skimage.feature import graycomatrix, graycoprops

# Transformeur qui renvoie le dataframe des caractéristiques GLCM
class GLCMExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, image_size=(60,110),
                 glcm_distances=[1],
                 glcm_angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                 glcm_props=['contrast','dissimilarity','homogeneity','energy','correlation']):
        self.image_size = image_size
        self.glcm_distances = glcm_distances
        self.glcm_angles = glcm_angles
        self.glcm_props = glcm_props
        # On détermine le nom des features GLCM à extraire
        self.feature_names_ = []
        # Pour chaque propriété, chaque distance et chaque angle
        for prop in self.glcm_props:
            for distance in self.glcm_distances:
                for angle in self.glcm_angles:
                    # On stocke dans les noms des features un label du type : contrast_d1_a45
                    self.feature_names_.append(f"{prop}_d{distance}_a{np.degrees(angle):.0f}")

    # Méthode d'extraction des caractéristiques GLCM d'une image
    def extract_glcm(self, gray_img):
        glcm_vector = []
        # Calcul de la matrice GLCM (256 x 256 x distances x angles)
        glcm = graycomatrix(
            gray_img,
            distances=self.glcm_distances,
            angles=self.glcm_angles,
            levels=256
        )
        # Pour chaque propriété GLCM
        for prop in self.glcm_props:
            # On la calcule pour les différentes distances et les différents angles 
            prop_matrix = graycoprops(glcm, prop)  # Matrice distances x angles
            # On transforme la matrice en vecteur qu'on stocke dans glcm_vector
            glcm_vector.extend(prop_matrix.flatten())
        return glcm_vector      

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # On va calculer les vecteurs GLCM
        glcm_vectors = []
        # On parcourt les chemins des images
        for img_path in X['Chemin']:
            # Lecture de l'image
            img = cv2.imread(img_path)
            # Conversion en niveaux de gris et resizing
            img_gray_resized = cv2.cvtColor(cv2.resize(img, self.image_size), cv2.COLOR_BGR2GRAY)
            # Extraction et stockage du vecteur GLCM
            glcm_vector = self.extract_glcm(img_gray_resized)
            glcm_vectors.append(glcm_vector)
        # On renvoie le dataframe des vecteurs GLCM
        return pd.DataFrame(glcm_vectors,index=X.index,columns=self.feature_names_)
    
    # Pour récupération du nom des features créées
    def get_feature_names_out(self, input_features=None):
        return self.feature_names_
    
# Création du transformeur
glcm_extr = GLCMExtractor()

#### Standardisation

In [8]:
# Création du StandardScaler
glcm_st = StandardScaler()

#### Création Pipeline GLCM

In [9]:
GLCMPipeline = Pipeline(steps=[
    ("Extraction GLCM",glcm_extr),
    ("Standardisation",glcm_st)
]) 

### Pipeline 3 : vecteurs Entropie

#### Transformeur pour extraction des vecteurs caractéristiques de l'entropie

In [10]:
from skimage.filters.rank import entropy
from skimage.morphology import disk
from scipy.stats import skew, kurtosis

# Transformeur qui renvoie le dataframe des caractéristiques de l'entropie
class EntropyExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, image_size=(60,110),radius=4, bins=10, include_histogram=True):
        self.image_size = image_size
        self.radius = radius
        self.bins = bins
        self.include_histogram = include_histogram
        # On détermine le nom des features Entropie à extraire
        self.feature_names_ = ['entropy_mean', 'entropy_std', 'entropy_min','entropy_max', 'entropy_median', 'entropy_skew', 'entropy_kurtosis']
        # Si histogramme demandé
        if include_histogram:
            self.feature_names_.extend([f'entropy_hist_bin{i+1}' for i in range(self.bins)])
        
    # Méthode d'extraction des caractéristiques de l'entropie d'une image
    def extract_entropy(self, gray_img):
        # Calcul de la carte d'entropie
        entropie = entropy(gray_img, disk(self.radius))
        # Calcul des statistiques associées
        entropy_vector = [
                np.mean(entropie),
                np.std(entropie),
                np.min(entropie),
                np.max(entropie),
                np.median(entropie),
                skew(entropie.ravel()),
                kurtosis(entropie.ravel())
            ]
        # Si histogramme demandé
        if (self.include_histogram):
            hist, bin_edges = np.histogram(entropie, bins=self.bins, range=(0, np.max(entropie)), density=True)
            # On l'ajoute au vecteur des caractéristiques de l'entropie
            entropy_vector.extend(list(hist))

        return entropy_vector      

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # On va calculer les vecteurs caractéristiques de l'entropie
        entropy_vectors = []
        # On parcourt les chemins des images
        for img_path in X['Chemin']:
            # Lecture de l'image
            img = cv2.imread(img_path)
            # Conversion en niveaux de gris et resizing
            img_gray_resized = cv2.cvtColor(cv2.resize(img, self.image_size), cv2.COLOR_BGR2GRAY)
            # Extraction et stockage du vecteur de l'entropie
            entropy_vector = self.extract_entropy(img_gray_resized)
            entropy_vectors.append(entropy_vector)
        # On renvoie le dataframe des vecteurs de l'entropie
        return pd.DataFrame(entropy_vectors,index=X.index,columns=self.feature_names_)
    
    # Pour récupération du nom des features créées
    def get_feature_names_out(self, input_features=None):
        return self.feature_names_
    
# Création du transformeur
entropy_extr = EntropyExtractor()

#### Standardisation

In [11]:
# Création du StandardScaler
entropy_st = StandardScaler()

#### Création pipeline Entropie

In [12]:
EntropyPipeline = Pipeline(steps=[
    ("Entropie",entropy_extr),
    ("Standardisation",entropy_st)
]) 

### Pipeline 4 : pixels bruts

#### Transformeur pour extraction des pixels bruts

In [13]:
# Transformeur qui renvoie le dataframe des pixels bruts
class PixelsBrutsExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, image_size=(30,55)):
        self.image_size = image_size
        # Nom des features : Pixel1, Pixel2...
        self.feature_names_ = [f"Pixel{i+1}" for i in range(image_size[0]*image_size[1])]  

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # On va calculer les vecteurs des pixels bruts
        brut_vectors = []
        # On parcourt les chemins des images
        for img_path in X['Chemin']:
            # Lecture de l'image
            img = cv2.imread(img_path)
            # Conversion en niveaux de gris et resizing
            img_gray_resized = cv2.cvtColor(cv2.resize(img, self.image_size), cv2.COLOR_BGR2GRAY)
            # Extraction et stockage du vecteur des pixels bruts
            brut_vector = img_gray_resized.flatten()
            brut_vectors.append(brut_vector)
        # On renvoie le dataframe des vecteurs des pixels bruts
        return pd.DataFrame(brut_vectors,index=X.index,columns=self.feature_names_)
    
    # Pour récupération du nom des features créées
    def get_feature_names_out(self, input_features=None):
        return self.feature_names_
    
# Création du transformeur
pixbrut_extr = PixelsBrutsExtractor()

#### Standardisation

In [14]:
# Création du StandardScaler
pixbrut_st = StandardScaler()

#### Création Pipeline Pixels Bruts

In [15]:
PixelsBrutsPipeline = Pipeline(steps=[
    ("Pixels bruts",pixbrut_extr),
    ("Standardisation",pixbrut_st)
]) 

### Pipelines finales avec modèle SVM

In [16]:
# Imports nécessaires
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC

# Modèle SVM
svm = SVC(kernel="rbf", gamma="scale", C=10)

# Feature extractors et scalers déjà définis
# -> hog_extr, hog_st, glcm_extr, glcm_st, entropy_extr, entropy_st, pixbrut_extr, pixbrut_st

# Pipelines corrigés
finalPipelinesSMOTE = {
    "Pixels Bruts + SVM + SMOTE": ImbPipeline(steps=[
        ('Pixels bruts', pixbrut_extr),
        ('Standardisation', pixbrut_st),
        ('SMOTE', SMOTE(random_state=42)),
        ('Modeling', svm)
    ]),

    "HOG + SVM + SMOTE": ImbPipeline(steps=[
        ('Extraction HOG', hog_extr),
        ('Standardisation', hog_st),
        ('SMOTE', SMOTE(random_state=42)),
        ('Modeling', svm)
    ]),

    "GLCM + SVM + SMOTE": ImbPipeline(steps=[
        ('Extraction GLCM', glcm_extr),
        ('Standardisation', glcm_st),
        ('SMOTE', SMOTE(random_state=42)),
        ('Modeling', svm)
    ]),

    "Entropie + SVM + SMOTE": ImbPipeline(steps=[
        ('Extraction Entropie', entropy_extr),
        ('Standardisation', entropy_st),
        ('SMOTE', SMOTE(random_state=42)),
        ('Modeling', svm)
    ]),

    "HOG/GLCM + SVM + SMOTE": ImbPipeline(steps=[
        ('Preprocessing', FeatureUnion([
            ('HOG', Pipeline([
                ('Extraction HOG', hog_extr),
                ('Standardisation', hog_st)
            ])),
            ('GLCM', Pipeline([
                ('Extraction GLCM', glcm_extr),
                ('Standardisation', glcm_st)
            ]))
        ])),
        ('SMOTE', SMOTE(random_state=42)),
        ('Modeling', svm)
    ]),

    "HOG/Entropie + SVM + SMOTE": ImbPipeline(steps=[
        ('Preprocessing', FeatureUnion([
            ('HOG', Pipeline([
                ('Extraction HOG', hog_extr),
                ('Standardisation', hog_st)
            ])),
            ('Entropie', Pipeline([
                ('Extraction Entropie', entropy_extr),
                ('Standardisation', entropy_st)
            ]))
        ])),
        ('SMOTE', SMOTE(random_state=42)),
        ('Modeling', svm)
    ]),

    "GLCM/Entropie + SVM + SMOTE": ImbPipeline(steps=[
        ('Preprocessing', FeatureUnion([
            ('GLCM', Pipeline([
                ('Extraction GLCM', glcm_extr),
                ('Standardisation', glcm_st)
            ])),
            ('Entropie', Pipeline([
                ('Extraction Entropie', entropy_extr),
                ('Standardisation', entropy_st)
            ]))
        ])),
        ('SMOTE', SMOTE(random_state=42)),
        ('Modeling', svm)
    ]),

    "HOG/GLCM/Entropie + SVM + SMOTE": ImbPipeline(steps=[
        ('Preprocessing', FeatureUnion([
            ('HOG', Pipeline([
                ('Extraction HOG', hog_extr),
                ('Standardisation', hog_st)
            ])),
            ('GLCM', Pipeline([
                ('Extraction GLCM', glcm_extr),
                ('Standardisation', glcm_st)
            ])),
            ('Entropie', Pipeline([
                ('Extraction Entropie', entropy_extr),
                ('Standardisation', entropy_st)
            ]))
        ])),
        ('SMOTE', SMOTE(random_state=42)),
        ('Modeling', svm)
    ]),
}

from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
from copy import deepcopy

def replace_sampler(pipelines, new_sampler_cls, suffix):
    """
    Duplique un dictionnaire de pipelines et remplace l'étape 'SMOTE'
    par new_sampler_cls(random_state=42). Renomme la clé pour refléter
    le nouveau sampler.
    """
    new_dict = {}
    for name, pipe in pipelines.items():
        steps = deepcopy(pipe.steps)           # copie profonde des étapes
        # cherche l'étape dont le nom contient 'SMOTE'
        for i, (step_name, step_obj) in enumerate(steps):
            if step_name.upper() == "SMOTE":
                steps[i] = (new_sampler_cls.__name__, new_sampler_cls(random_state=42))
                break
        new_name = name.replace("SMOTE", suffix)
        new_dict[new_name] = ImbPipeline(steps)
    return new_dict

# Dictionnaire pour SMOTE-Tomek
finalPipelines_SMOTETomek = replace_sampler(finalPipelinesSMOTE, SMOTETomek, "SMOTETomek")

# Dictionnaire pour ADASYN
finalPipelines_ADASYN = replace_sampler(finalPipelinesSMOTE, ADASYN, "ADASYN")


### Entraînement et évaluation

In [ ]:
# ============================================================
# 0) IMPORTS
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    f1_score, make_scorer, accuracy_score,
    confusion_matrix, classification_report
)

# ------------------------------------------------------------------
# *Prérequis* : les objets suivants doivent déjà exister dans le notebook
#   X_train, y_train, X_test, y_test                       (pandas ou numpy)
#   finalPipelinesSMOTE, finalPipelines_SMOTETomek, finalPipelines_ADASYN
# ------------------------------------------------------------------


# ============================================================
# 1) PARAMÈTRES DU CLASSIFIEUR  (modifiez ici)
# ============================================================
SVM_C        = 100             # rigidité (1–10 conseillé au début)
SVM_GAMMA    = 0.01           # "scale", 0.1, 0.01 …
EXTRA_PARAMS = dict(class_weight="balanced")   # {} si vous ne voulez rien

# Validation croisée
N_SPLITS     = 10             # nb de folds
RANDOM_STATE = 42

# Classes rares
minority_labels = ["break", "string short circuit"]

# Palette couleurs pour les graphiques
palette = {"SMOTE": "#4c72b0", "SMOTE-Tomek": "#55a868", "ADASYN": "#c44e52"}


# ============================================================
# 2) SCORER MINORITAIRE
# ============================================================
def minority_f1_safe(y_true, y_pred, labels=minority_labels):
    present = [lab for lab in labels if lab in y_true.values]
    if not present:
        return 0.0
    return f1_score(y_true, y_pred, labels=present,
                    average="macro", zero_division=0)

minority_f1 = make_scorer(minority_f1_safe, greater_is_better=True)


# ============================================================
# 3) STRATIFIED K-FOLD GARANTISSANT LES CLASSES RARES
# ============================================================
def safe_stratified_splits(y, labels, n_splits=N_SPLITS,
                           max_attempts=1000, seed=RANDOM_STATE):
    """Retourne une liste (train_idx, val_idx) où chaque val contient ≥1 ex. de chaque label minoritaire."""
    rng = np.random.RandomState(seed)
    for attempt in range(max_attempts):
        skf = StratifiedKFold(
            n_splits=n_splits, shuffle=True,
            random_state=rng.randint(0, 1_000_000)
        )
        splits = list(skf.split(np.zeros(len(y)), y))
        if all(
            all((y.iloc[val] == lab).sum() > 0 for lab in labels)
            for _, val in splits
        ):
            print(f"✓ Splits trouvés après {attempt+1} tentative(s)")
            return splits
    raise RuntimeError("Impossible de satisfaire la contrainte sur les classes rares.")

safe_splits = safe_stratified_splits(y_train, minority_labels)


# ============================================================
# 4) BOUCLE D’ÉVALUATION
# ============================================================
def evaluate_family(pipelines: dict, sampler_tag: str, store: list):
    for pipe_name, pipe in pipelines.items():
        # -- fixe les hyper-paramètres du SVM --
        svm_key = list(pipe.named_steps.keys())[-1]
        pipe.named_steps[svm_key].set_params(
            C=SVM_C, gamma=SVM_GAMMA, **EXTRA_PARAMS
        )

        # -- fit + prédiction test --
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        store.append({
            "Sampler":       sampler_tag,
            "Pipeline":      pipe_name,
            "Accuracy":      accuracy_score(y_test, y_pred),
            "Macro_F1":      f1_score(y_test, y_pred, average="macro", zero_division=0),
            "Minority_F1":   minority_f1_safe(y_test, y_pred),

            "CV_Minority_F1": cross_val_score(
                pipe, X_train, y_train,
                scoring=minority_f1,
                cv=safe_splits,
                n_jobs=-1
            ).mean()
        })


# ============================================================
# 5) EXÉCUTION SUR LES 3 FAMILLES DE PIPELINES
# ============================================================
results = []
evaluate_family(finalPipelinesSMOTE,       "SMOTE",       results)
evaluate_family(finalPipelines_SMOTETomek, "SMOTE-Tomek", results)
evaluate_family(finalPipelines_ADASYN,     "ADASYN",      results)

df_scores = pd.DataFrame(results)


# ============================================================
# 6) VISUALISATIONS
# ============================================================
sns.set_style("whitegrid")

# (A) Minority-F1 par pipeline et sampler
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(df_scores))
ax.bar(x, df_scores["Minority_F1"],
       color=[palette[s] for s in df_scores["Sampler"]])
ax.set_xticks(x)
ax.set_xticklabels(df_scores["Pipeline"], rotation=90, ha="center", fontsize=8)
ax.set_ylabel("Minority-F1 (test)")
ax.set_title("Impact de l’oversampling sur les classes minoritaires")
for lab, col in palette.items():
    ax.bar(0, 0, color=col, label=lab)
ax.legend(); plt.tight_layout(); plt.show()

# (B) Moyenne globale par technique d’oversampling
mean_df = (df_scores
           .groupby("Sampler")[["Minority_F1", "Macro_F1", "Accuracy"]]
           .mean()
           .reset_index())

fig, ax = plt.subplots(figsize=(5, 4))
ax.bar(mean_df["Sampler"], mean_df["Minority_F1"],
       color=[palette[s] for s in mean_df["Sampler"]])
ax.set_ylabel("Minority-F1 moyen (test)")
ax.set_title("F1 minoritaire moyen par technique d’oversampling")
plt.show()

print("=== Tableau récapitulatif ===")
display(df_scores.round(3))



✓ Splits trouvés après 1 tentative(s)


### TODO

- Preprocessing : faire varier les paramètres des transformeurs (extracteurs + standardisation)
- Preprocessing : intégrer rééchantillonnage (ou 'balanced'), intégrer réduction de dimension
- Comparer l'importance des features (permutation_importance...)
- Comparer les résultats si on retire en entrée les images 'Doute_Carre'
- Essayer d'autres modèles : KNN, RandomForest, MLPClassifier, modèles d'ensemble
- Question : part-on dès maintenant sur un ensemble train / val / test commun à tout le monde ?
    - Pour chaque modèle, ajuster ses hyperparamètres sur l'ensemble train par validation croisée (GridSearch,...)
    - Entraîner tous les modèles optimisés sur l'ensemble train et comparer leur résultat sur l'ensemble val
    - Entraîner le meilleur modèle sur train + val, et vérifier sa bonne généralisation sur test.